## Data Collection Phase 

You have to scrape at least 20000 rows of data. You can scrape more data as well, it’s up to you. More the data better the model. In this section you need to scrape the reviews of different laptops, Phones, Headphones, smartwatches, Professional Cameras, Printers, Monitors, Home theater, Router from different ecommerce websites.

Basically, we need these columns:
1) reviews of the product.
2) rating of the product.
You can fetch other data as well, if you think data can be useful or can help in the project. It completely depends on your imagination or assumption.

In [1]:
# Import all the neccessary libraries

import pandas as pd
from bs4 import BeautifulSoup
import requests
import selenium
from selenium import webdriver
import time
import warnings                                            
warnings.filterwarnings('ignore')

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException,ElementNotInteractableException,WebDriverException,StaleElementReferenceException 

### Reviews from Flipkart.com

In [2]:
driver = webdriver.Chrome(r"C:\Users\jyuth\Chrome driver 103\chromedriver.exe")
url = "https://www.flipkart.com/"
driver.get(url)
time.sleep(2)

#clicking on cancel button 
driver.find_element_by_xpath("//div[@class='_2QfC02']/button").click()

srch_items = ['laptops','Phones','Headphones','smart watches', 'Professional Cameras','Printers','monitors',
              'Home theater','router']
title = []
review_text = []
ratings = []

In [3]:
def scrap():    
        for i in driver.find_elements_by_xpath("//div[@class='t-ZTKy']"):  #Scrapping review text
            review_text.append(i.text)
        for i in driver.find_elements_by_xpath("//p[@class='_2-N8zT']"): #scrapping title of the review
            title.append(i.text)
        for i in driver.find_elements_by_xpath("//p[@class='_2-N8zT']/../div"):  #scrapping the stars
            ratings.append(i.text)
        return
urls=[]
for i in srch_items:
    # Find the search bar
    srchBar = driver.find_element_by_xpath("//div[@class='_3OO5Xc']/input")
    srchBar.clear()
    srchBar.send_keys(i)
    
    # Clicking on the search button
    driver.find_element_by_xpath("//button[@class='L0Z3Pu']").click()
    time.sleep(3)
    page = []
    for i in driver.find_elements_by_xpath("//nav[@class='yFHi8N']/a"):
        page.append(i.get_attribute('href'))
    for i in page[0:4]:
        driver.get(i)
        time.sleep(3)
        items = driver.find_elements_by_xpath("//a[@class='_1fQZEK']")
        for i in items:
            urls.append(i.get_attribute('href'))
len(urls)            

288

In [4]:
for i in urls:
    driver.get(i)
    time.sleep(2)
    for _ in range(2):
        driver.execute_script("window.scrollBy(0,6000)")
        time.sleep(1)
        
    # Clicking on all reviews
    try:
        btn=driver.find_element_by_xpath("//div[@class='_2c2kV-']/following::a")
        lnk = btn.get_attribute('href')
        driver.get(lnk)
        time.sleep(1)
    except NoSuchElementException:
        pass

    scrap()        
    try:
        n_page=driver.find_elements_by_xpath("//nav[@class='yFHi8N']/a")
        np=[]
        for i in n_page:
            np.append(i.get_attribute('href'))
        for i in np[0:18]:
            driver.get(i)
            time.sleep(1)
            scrap()
    except: continue
len(ratings), len(review_text), len(title)        

(21444, 21444, 21444)

In [5]:
# Creating a dataframe
data = list(zip(title,review_text,ratings))
df = pd.DataFrame(data, columns = ["Review_title","Review_text","Ratings"])
df

,Review_title,Review_text,Ratings
0,Fabulous!,After one month of usage for office work and m...,5
1,Worth every penny,Laptop is worth the price. It is fast and smoo...,5
2,Great product,I got it for 54k\nFor this price\nBuild Qualit...,5
3,Great product,One of the best laptop in given prize segment....,5
4,Awesome,Value for money just go for it✨,5
...,...,...,...
21439,Brilliant,Very nice,5
21440,Excellent,Excellent Resolution\nGood working\nVery very ...,5
21441,Brilliant,Really good monitor. Very effective and useful...,5
21442,Good choice,Actually its not a 24 inch display its just 21...,4


In [6]:
# Saving the data into csv file
df.to_csv('Rating.csv')

In [ ]:
driver.close()